In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model='Gemma2-9b-It', groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002B6ECB26320>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002B6ECB501C0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi. My name is Sushravya and I am a Gen AI enthusiast.")])

AIMessage(content="Hi Sushravya,\n\nIt's great to meet you! I'm glad you're interested in Generative AI. It's a fascinating field with a lot of potential.  \n\nWhat aspects of Generative AI particularly interest you? Are you working on any projects yourself? I'd love to hear more about your enthusiasm!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 25, 'total_tokens': 98, 'completion_time': 0.132727273, 'prompt_time': 0.000169729, 'queue_time': 0.01400598, 'total_time': 0.132897002}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-74f79e31-281e-4ad4-b099-cf60ff607a2d-0', usage_metadata={'input_tokens': 25, 'output_tokens': 73, 'total_tokens': 98})

In [7]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi. My name is Sushravya and I am a Gen AI enthusiast."),
        AIMessage(content="Hi Sushravya,\n\nIt's great to meet you! I'm glad you're interested in Generative AI. It's a fascinating field with a lot of potential.  \n\nWhat aspects of Generative AI particularly interest you? Are you working on any projects yourself? I'd love to hear more about your enthusiasm!"),
        HumanMessage(content="Hey. What's my name and what am I interested in?")
    ]
)

AIMessage(content="You said your name is Sushravya and that you are a Gen AI enthusiast!  😊 \n\nIs there anything else you'd like to tell me about yourself or ask me? I'm here to chat. 🤖  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 119, 'total_tokens': 171, 'completion_time': 0.094545455, 'prompt_time': 0.00345661, 'queue_time': 0.011990586999999999, 'total_time': 0.098002065}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8607959f-0c61-4413-89e4-0c4ce122856c-0', usage_metadata={'input_tokens': 119, 'output_tokens': 52, 'total_tokens': 171})

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [11]:
config = {"configurable" : {"session_id" : "chat1"}}

In [12]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi. My name is Sushravya and I am a Gen AI enthusiast.")],
    config=config
)

In [13]:
response.content

"Hi Sushravya, it's nice to meet you!\n\nI'm glad to hear you're a Gen AI enthusiast. It's an exciting field with a lot of potential. What are you most interested in about Gen AI?  \n\nI'm always happy to chat about it!\n"

In [14]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

AIMessage(content='Your name is Sushravya. 😊  \n\nI remember that from our introduction!  Is there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 104, 'total_tokens': 135, 'completion_time': 0.056363636, 'prompt_time': 0.003478621, 'queue_time': 0.009753478000000001, 'total_time': 0.059842257}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9c1df01e-1375-4f73-baf1-1b29c380ec71-0', usage_metadata={'input_tokens': 104, 'output_tokens': 31, 'total_tokens': 135})

In [15]:
# Change the config --> session id
config1 = {"configurable" : {"session_id" : "chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config1
)
response.content

"As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to know!\n"

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="Hey. My name is John")],
    config=config1
)
response.content

"Hi John!\n\nIt's nice to meet you. 😊  What can I do for you today?\n"

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")], config=config1
)
response.content

'Your name is John. I remember! 😊  \n\nAnything else I can help you with?\n'

#### Prompt Templates

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [21]:
chain.invoke({"messages": [HumanMessage(content="Hi. My name is Sushravya.")]})

AIMessage(content="Hello Sushravya, it's nice to meet you! How can I be of help today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 34, 'total_tokens': 61, 'completion_time': 0.049090909, 'prompt_time': 0.001612956, 'queue_time': 0.012521792, 'total_time': 0.050703865}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-44807c73-2cc0-44a6-b603-f66c8a86cb71-0', usage_metadata={'input_tokens': 34, 'output_tokens': 27, 'total_tokens': 61})

In [22]:
config = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi My name is Sushravya")], config=config
)

response

AIMessage(content="Hi Sushravya, it's nice to meet you! What can I do for you today? 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 16, 'total_tokens': 41, 'completion_time': 0.045454545, 'prompt_time': 8.71e-05, 'queue_time': 0.015371878, 'total_time': 0.045541645}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-30c9508b-9155-42fd-b3e3-aa15b14c415b-0', usage_metadata={'input_tokens': 16, 'output_tokens': 25, 'total_tokens': 41})

In [23]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Sushravya.  I remember that! 😊  \n\nIs there anything else I can help you with?\n'

In [24]:
# Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt | model

In [25]:
response = chain.invoke({"messages":[HumanMessage(content="Hi. My name is Sushravya.")], "language" : "Kannada"})
response.content

'ನಮಸ್ಕಾರ Sushravya! 😊\n\nನನ್ನ ಹೆಸರು Kannada Assistant. ನಾನು ನಿಮಗಾಗಿ ಎಲ್ಲಾ ಪ್ರಶ್ನೆಗಳಿಗೆ ಸಹಕರಿಸಲು ನಿತ್ಯ ಸಿದ್ಧನಾಗಿರುತ್ತೇನೆ.  \n\nನಿಮಗೆ ಏನು ಕೇಳ್ಕೊಳ್ಳಬೇಕು? 🤔\n\n'

In [26]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [27]:
config = {"configurable" : {"session_id" : "chat4"}}
response = with_message_history.invoke(
    {'messages' : [HumanMessage(content="Hi. I am Sushravya")], "language" : "Kannada"},
    config = config
)
response.content

'ನಮಸ್ಕಾರ, ಸುಶ್ರಾವ್ಯ! 😊  \n\nನನ್ನ ಹೆಸರು ಒಂದು ಸಹಕಾರಿ ಸಾಫ್ಟ್\u200cವೇರ್ ಅನುಕೂಲ ಅನ್ನು. ನಾನು ನಿಮ್ಮ ಎಲ್ಲಾ ಪ್ರಶ್ನೆಗಳಿಗೆ ಸಹಾಯ ಮಾಡಲು ಬಯಸುತ್ತೇನೆ. \n\nನಿಮಗೆ ಏನು ಕೇಳಬೇಕು? \n\n'

In [28]:
response = with_message_history.invoke(
    {"messages" : [HumanMessage(content="what's my name?")], "language" : "Kannada"},
    config=config,
)

In [29]:
response.content

'ನಿಮ್ಮ ಹೆಸರು ಸುಶ್ರಾವ್ಯ. 😊 \n'

In [32]:
from langchain_core.messages import SystemMessage, trim_messages
from sympy import content
trimmer = trim_messages(
    max_tokens = 80,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

messages = [
    SystemMessage(content="You're a good assistant."),
    HumanMessage(content="Hi! I'm Lily."),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like vanilla ice cream."),
    AIMessage(content="Nice"),
    HumanMessage(content="What's 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks!"),
    AIMessage(content="No problem."),
    HumanMessage(content="Having fun?"),
    AIMessage(content="Yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="You're a good assistant.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Hi! I'm Lily.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's 2 + 2", additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes!', additional_kwargs={}, response_metadata={})]

In [33]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages" : messages + [HumanMessage(content="What ice cream do I like?")],
        "language" : "English"
    }
)
response.content

'You like vanilla ice cream!  🍦'

In [34]:
response = chain.invoke(
    {
        "messages" : messages + [HumanMessage(content="What math problem did I ask?")],
        "language" : "English",
    }
)
response.content

"You asked: What's 2 + 2?\n"

In [35]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable" : {"session_id" : "chat5"}}

In [37]:
response = with_message_history.invoke(
    {
        "messages" : messages + [HumanMessage(content="What's my name")],
        "language" : "English",
    },
    config=config
)
response.content

'Your name is Lily. 😊  \n\n\n\n'

In [38]:
response = with_message_history.invoke(
    {
        "messages" : [HumanMessage(content="What math problem did I ask?")],
        "language" : "English",
    },
    config=config,
)
response.content

'You asked "What\'s 2 + 2"  🍦'